## Data Exploration

In [1]:
# import all the neccessary libraries
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
cc_data = pd.read_csv("cervical-cancer_csv.csv")

In [ ]:
cc_data.head()